# Power laws from intrinsic dendritic filtering

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.fftpack as ff
from elephant.spike_train_generation import homogeneous_poisson_process
from quantities import s, Hz, ms
import neuron
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
import brainsignals.neural_simulations as ns

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(1534)

In [ ]:
sigma = 0.3
cut_off = 200
tstop = 2**12 + cut_off
dt = 2**-6
rate = 5000 * Hz

synapse_params = {
    'syntype' : 'ExpSynI',      #current based exponential synapse
    'tau' : dt / 3,                #Time constant
    'weight' : 0.01,           #Synaptic weight
    'record_current' : False,    #record synaptic currents
}

num_elecs = 2
# Define electrode parameters
elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': np.array([1, 10000 * np.cos(np.deg2rad(-30))]),  # electrode positions
    'y': np.zeros(num_elecs),
    'z': np.array([0, 10000 * np.sin(np.deg2rad(-30))]),
    'method': 'pointsource'
}


def insert_synapses(cell, synapse_params, syn_idxs,  spiketimes):
    for num, idx in enumerate(syn_idxs):
        synapse_params.update({'idx' : idx})
        s = LFPy.Synapse(cell, **synapse_params)
        s.set_spike_times(spiketimes[num])

        
def make_white_noise_stimuli(cell, input_idx, freqs, tvec, input_scaling=0.005):

    I = np.zeros(len(tvec))

    for freq in freqs:
        I += np.sin(2 * np.pi * freq * tvec/1000. + 2*np.pi*np.random.random())    
    input_array = input_scaling * I

    noise_vec = neuron.h.Vector(input_array)

    i = 0
    syn = None
    for sec in cell.allseclist:
        for seg in sec:
            if i == input_idx:
                #print("Input inserted in ", sec.name())
                syn = neuron.h.ISyn(seg.x, sec=sec)
            i += 1
    if syn is None:
        raise RuntimeError("Wrong stimuli index")
    syn.dur = 1E9
    syn.delay = 0
    noise_vec.play(syn._ref_amp, cell.dt)
    return cell, syn, noise_vec        


In [ ]:

input_idx = 0

num_tsteps = int(tstop / dt + 1)
tvec = np.arange(num_tsteps) * dt

t0_idx = np.argmin(np.abs(tvec - cut_off))

divide_into_welch = 16
welch_dict = {'Fs': 1000 / dt,
              'NFFT': int(num_tsteps/divide_into_welch),
              'noverlap': int(num_tsteps/divide_into_welch/2.),
              'detrend': 'mean',
              'scale_by_freq': True,
              }


sample_freq = ff.fftfreq(num_tsteps, d=dt / 1000)
pidxs = np.where(sample_freq >= 0)
freqs = sample_freq[pidxs]

cell = ns.return_ball_and_stick_cell(tstop=tstop, dt=dt, apic_diam=2)

st_0 = np.array(homogeneous_poisson_process(rate, t_stop=tstop * ms))
insert_synapses(cell, synapse_params, [0], [st_0])

cell.simulate(rec_imem=True, rec_vmem=True)

cell.vmem = cell.vmem[:, t0_idx:]
cell.imem = cell.imem[:, t0_idx:]
cell.tvec = cell.tvec[t0_idx:] - cell.tvec[t0_idx]

electrode = LFPy.RecExtElectrode(cell, **elec_params)
lfp = electrode.get_transformation_matrix() @ cell.imem * 1000
cdm = LFPy.CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem
cdm = cdm[2, :]

freqs_s, psd_lfp_s = ns.return_freq_and_psd_welch(lfp, welch_dict)
freqs_s, psd_soma_i_s = ns.return_freq_and_psd_welch(cell.imem[0, :], welch_dict)
freqs_s, psd_cdm_s = ns.return_freq_and_psd_welch(cdm, welch_dict)

del cell

In [ ]:

cell = ns.return_ball_and_stick_cell(tstop=tstop, dt=dt)
syn_list = []
vec_list = []
print("inserting synapse currents")
for input_idx in range(cell.totnsegs):
    st_0 = np.array(homogeneous_poisson_process(rate, t_stop=tstop * ms))
    insert_synapses(cell, synapse_params, [input_idx], [st_0])

print("simulating")
cell.simulate(rec_imem=True, rec_vmem=True)

cell.vmem = cell.vmem[:, t0_idx:]
cell.imem = cell.imem[:, t0_idx:]
cell.tvec = cell.tvec[t0_idx:] - cell.tvec[t0_idx]
print("Max vmem: ", np.max(cell.vmem))

electrode = LFPy.RecExtElectrode(cell, **elec_params)
lfp = electrode.get_transformation_matrix() @ cell.imem * 1000
print("Max LFP: ", np.max(np.abs(lfp)))

cdm = LFPy.CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem
cdm = cdm[2, :]

freqs_u, psd_lfp_u = ns.return_freq_and_psd_welch(lfp, welch_dict)
freqs_u, psd_soma_i_u = ns.return_freq_and_psd_welch(cell.imem[0, :], welch_dict)
freqs_u, psd_cdm_u = ns.return_freq_and_psd_welch(cdm, welch_dict)


In [ ]:
plt.close("all")
fig = plt.figure()
fig.subplots_adjust(bottom=0.3)

ax_s = fig.add_subplot(121, xlabel="frequency (Hz)", ylabel="[a.u.]²/Hz",
                      xlim=[1, 2e4], ylim=[1e-5, 1e1],  xscale="log",
                      title="somatic input")
ax_u = fig.add_subplot(122, xlabel="frequency (Hz)", ylabel="[a.u.]²/Hz",
                      xlim=[1, 2e4], ylim=[1e-5, 1e1], xscale="log",
                      title="uniform input")

l1, = ax_s.loglog(freqs_s[1:], psd_lfp_s[0, 1:] / psd_lfp_s[0, 1], c='k', lw=2)
l2, = ax_s.loglog(freqs_s[1:], psd_lfp_s[1, 1:] / psd_lfp_s[1, 1], c='r', lw=2)
l3, = ax_s.loglog(freqs_s[1:], psd_soma_i_s[0, 1:] / psd_soma_i_s[0, 1], c='orange', lw=1.)
l4, = ax_s.loglog(freqs_s[1:], psd_cdm_s[0, 1:] / psd_cdm_s[0, 1], c='pink', lw=1.)

l1, = ax_u.loglog(freqs_u[1:], psd_lfp_u[0, 1:] / psd_lfp_u[0, 1], c='k', lw=2)
l2, = ax_u.loglog(freqs_u[1:], psd_lfp_u[1, 1:] / psd_lfp_u[1, 1], c='r', lw=2)
l3, = ax_u.loglog(freqs_u[1:], psd_soma_i_u[0, 1:] / psd_soma_i_u[0, 1], c='orange', lw=1.)
l4, = ax_u.loglog(freqs_u[1:], psd_cdm_u[0, 1:] / psd_cdm_u[0, 1], c='pink', lw=1.)

f_ = np.array([2e3, 2e4])
f_idx = np.argmin(np.abs(f_[0] - freqs))

y_0 = psd_lfp_s[0, f_idx] / psd_lfp_s[0, 1] * (f_[0] / f_)**1 * 30
y_1 = psd_lfp_s[1, f_idx] / psd_lfp_s[1, 1] * (f_[0] / f_)**2 * 100

ax_s.loglog(f_, y_0, 'gray', lw=2)
ax_s.loglog(f_, y_1, 'gray', lw=2)

ax_s.text(2e4, 2e-2, "1/$f$", color="gray")
ax_s.text(2e4, 1e-4, "1/$f^2$", color="gray")

f_ = np.array([2e3, 2e4])
f_idx = np.argmin(np.abs(f_[0] - freqs_u))

y_0 = psd_lfp_u[0, f_idx] / psd_lfp_u[0, 1] * (f_[0] / f_)**(1/2) * 3
y_1 = psd_lfp_u[1, f_idx] / psd_lfp_u[1, 1] * (f_[0] / f_)**(3/2) * 1

ax_u.loglog(f_, y_0, 'gray', lw=2)
ax_u.loglog(f_, y_1, 'gray', lw=2)

ax_u.text(2e4, 5e-1, "1/$f^{1/2}$", color="gray")
ax_u.text(2e4, 2e-4, "1/$f^{3/2}$", color="gray")

ax_s.set_xticks([1, 10, 100, 1000, 10000])
ax_u.set_xticks([1, 10, 100, 1000, 10000])

ax_s.grid(True)
ax_u.grid(True)

fig.legend([l1, l2, l3, l4], 
           [r"|$V_{\rm e}(10~$µm$)$|", 
            r"|$V_{\rm e}(10~000~$µm$)$|", 
            r"$I_{\rm soma}$", "$P_z$"], frameon=False, ncol=4,
          loc="lower center")

simplify_axes(fig.axes)
mark_subplots(fig.axes)

locmin = matplotlib.ticker.LogLocator(base=10.0,
                                      subs=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9),
                                      numticks=12)
ax_u.xaxis.set_minor_locator(locmin)
ax_u.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

locmin = matplotlib.ticker.LogLocator(base=10.0,
                                      subs=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9),
                                      numticks=12)
ax_s.xaxis.set_minor_locator(locmin)
ax_s.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())


fig.savefig("fig_LFP_powerlaws_intrinsic.pdf")